<a href="https://colab.research.google.com/github/snpushpi/Projects_with_Tensor2tensor/blob/master/Lstm_Automatic_Speech_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U tensor2tensor
!pip install -q tensorflow matplotlib
!pip install tensorflow==1.14.0
!pip install tensor2tensor==1.11.0
!pip install tensorflow-probability==0.7.0
!pip install pydub
!apt-get -qq update
!apt-get -qq install -y ffmpeg
!apt-get -qq install -y sox

ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
  Using cached https://files.pythonhosted.org/packages/d3/59/d88fe8c58ffb66aca21d03c0e290cd68327cc133591130c674985e98a482/tensorflow-1.14.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/f4/37/e6a7af1c92c5b68fb427f853b06164b56ea92126bcfd87784334ec5e4d42/tensorboard-1.14.0-py2-none-any.whl
  Using cached https://files.pythonhosted.org/packages/3c/d5/21860a5b11caf0678fbc8319341b0ae21a07156911132e0e71bffed0510d/tensorflow_estimator-1.14.0-py2.py3-none-any.whl
ERROR: tensorflow-addons 0.7.1 has requirement tensorflow>=2.1.0, but you'll have tensorflow 1.14.0 which is incompatible.
  Found existing installation: tensorboard 2.1.0
    Uninstalling tensorboard-2.1.0:
      Successfully uninstalled tensorboard-2.1.0
  Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.1.0:
      Successfully uninstal

# New Section

In [2]:
import sys
if 'google.colab' in sys.modules: # Colab-only TensorFlow version selector
  %tensorflow_version 1.x
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import collections
import base64
import cStringIO
import pydub
import shutil
from scipy.io import wavfile

import IPython
import google.colab

from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

# Enable TF Eager execution
tfe = tf.contrib.eager
tf.enable_eager_execution()

# Other setup
Modes = tf.estimator.ModeKeys

# Setup some directories
data_dir = os.path.expanduser("~/t2t/data")
tmp_dir = os.path.expanduser("~/t2t/tmp")
train_dir = os.path.expanduser("~/t2t/train")
tf.gfile.MakeDirs(data_dir)
tf.gfile.MakeDirs(tmp_dir)
tf.gfile.MakeDirs(train_dir)

TensorFlow 1.x selected.


W0728 16:29:09.052959 139747548997504 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensor2tensor/utils/expert_utils.py:68: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0728 16:29:10.486263 139747548997504 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0728 16:29:12.297512 139747548997504 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensor2tensor/utils/adafactor.py:27: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0728 16:29:12.298140 139747548997504 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/d

In [3]:
problem_name = "librispeech_clean"
asr_problem = problems.problem(problem_name)
encoders = asr_problem.feature_encoders(None) 

model_name = "lstm_seq2seq"
hparams_set = "lstm_asr_v1"
t2t_problem = problems.problem(problem_name)
t2t_problem.generate_data(data_dir, tmp_dir)




W0728 16:29:20.785892 139747548997504 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensor2tensor/data_generators/generator_utils.py:129: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

W0728 16:29:20.794538 139747548997504 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensor2tensor/data_generators/generator_utils.py:163: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W0728 16:29:20.796180 139747548997504 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensor2tensor/data_generators/generator_utils.py:225: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

W0728 16:29:20.797192 139747548997504 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensor2tensor/data_generators/generator_utils.py:231: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.


W0728 16:29:36.307993 139747548997504 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensor2tensor/data_generators/generator_utils.py:241: The name tf.gfile.Rename is deprecated. Please use tf.io.gfile.rename instead.



100% completed
100% completed
100% completed


IOError: ignored

In [5]:
train_steps = 300000 # Total number of train steps for all Epochs
eval_steps = 100 # Number of steps to perform for each evaluation
batch_size = 20
save_checkpoints_steps = 1000
ALPHA = 0.2
schedule = "continuous_train_and_eval"

In [6]:
hparams = trainer_lib.create_hparams(hparams_set,data_dir=data_dir, problem_name=problem_name)

In [7]:
hparams.batch_size = 20

In [8]:
hparams.learning_rate = 0.2

Model Run

In [16]:
RUN_CONFIG = trainer_lib.create_run_config(
      model_dir=train_dir,
      model_name=model_name,
      save_checkpoints_steps= save_checkpoints_steps
)

tensorflow_exp_fn = trainer_lib.create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        model_name=model_name,
        problem_name=problem_name,
        data_dir=data_dir, 
        train_steps=train_steps, 
        eval_steps=eval_steps, 
        #use_xla=True # For acceleration
        use_tpu = True
    ) 

tensorflow_exp_fn.train_and_evaluate()

W0728 16:10:04.360632 139905117202304 devices.py:141] Schedule=continuous_train_and_eval. Assuming that training is running on a single machine.


ValueError: ignored

In [9]:
ckpt_path = tf.train.latest_checkpoint(os.path.join(train_dir))

In [ ]:
import json

In [10]:
asr_model = registry.model(model_name)(hparams, Modes.PREDICT)

def encode(x):
  waveforms = encoders["waveforms"].encode(x)
  encoded_dict = asr_problem.preprocess_example({"waveforms":waveforms, "targets":[]}, Modes.PREDICT, hparams)
  
  return {"inputs" : tf.expand_dims(encoded_dict["inputs"], 0), "targets" : tf.expand_dims(encoded_dict["targets"], 0)}

def decode(integers):
  integers = list(np.squeeze(integers))
  if 1 in integers:
    integers = integers[:integers.index(1)]
  return encoders["targets"].decode(np.squeeze(integers))

W0728 17:54:51.334754 139747548997504 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensor2tensor/layers/common_layers.py:75: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0728 17:54:51.747086 139747548997504 deprecation_wrapper.py:119] From /usr/local/lib/python2.7/dist-packages/tensor2tensor/utils/t2t_model.py:167: The name tf.summary.text is deprecated. Please use tf.compat.v1.summary.text instead.



In [11]:
# Restore and transcribe!
def transcribe(inputs):
  encoded_inputs = encode(inputs)
  with tfe.restore_variables_on_create(ckpt_path): 
    model_output = asr_model.infer(encoded_inputs, beam_size=2, alpha=0.6, decode_length=1)["outputs"]
  return decode(model_output)

def play_and_transcribe(inputs):
  waveforms = encoders["waveforms"].encode(inputs)
  IPython.display.display(IPython.display.Audio(data=waveforms, rate=16000))
  return transcribe(inputs) 

Record your own voice

In [14]:
# Records webm file and converts
def RecordNewAudioSample(filename=None, webm_filename=None):
  """Args:
          filename - string, path for storing wav file
          webm_filename - string, path for storing webm file
     Returns:
          string - path where wav file was saved. (=filename if specified)
    
  """
  # Create default filenames in tmp_dir if not specified.
  if not filename:
    filename = os.path.join(tmp_dir, "recording.wav")
  if not webm_filename:
    webm_filename = os.path.join(tmp_dir, "recording.webm")
    
  # Record webm file form colab.
  
  audio = google.colab._message.blocking_request('user_media', {"audio":True, "video":False, "duration":-1}, timeout_sec=600)
  #audio = frontend.RecordMedia(True, False)
  
  # Convert the recording into in_memory file.
  music_mem_file = cStringIO.StringIO(
      base64.decodestring(audio[audio.index(',')+1:]))
  
  # Store webm recording in webm_filename. Storing is necessary for conversion.
  with open(webm_filename, 'w') as fd:
    music_mem_file.seek(0)
    shutil.copyfileobj(music_mem_file, fd)
    
  # Open stored file and save it as wav with sample_rate=16000.
  pydub.AudioSegment.from_file(webm_filename, codec="opus"
                              ).set_frame_rate(16000).export(out_f=filename,
                                                             format="wav")
  return filename

Talk here talk here ! Click the cell next to this text :)

In [15]:
# Record the sample
my_sample_filename = RecordNewAudioSample()
print my_sample_filename

IOError: ignored

Now Click it and see what you said :)

In [ ]:
print play_and_transcribe(my_sample_filename)

Upload pre-recorded samples

In [12]:
uploaded = google.colab.files.upload()
prerecorded_messages = []

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  mem_file = cStringIO.StringIO(uploaded[fn])
    
  save_filename = os.path.join(tmp_dir, fn)
  with open(save_filename, 'w') as fd:
    mem_file.seek(0)
    shutil.copyfileobj(mem_file, fd)
  prerecorded_messages.append(save_filename)
         
              
for inputs in prerecorded_messages:
  outputs = play_and_transcribe(inputs)

  print("Inputs: %s" % inputs)
  print("Outputs: %s" % outputs)

Saving voice7.wav to voice7.wav


IOError: ignored